## Калькуляция максимально возможного значения метрики Precision@10.

Здесь мы рассчитываем максимально возможное значения `Precision@10` для тестовой выборки пользователей.

In [ ]:
# Импортируем основные библиотеки для работы с данными и Torch
import pandas as pd
import numpy as np
import random
import torch
from tqdm import tqdm
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Установка детерминированного поведения для воспроизводимости результатов
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [3]:
# Загружаем тестовый датасет и обучающую выборку (rating-based sampling)
# для последующего формирования ground truth
df_test = pd.read_csv("data/df_test_ground_truth_rating_based.csv")

In [4]:
# Создаём словарь ground truth: реальные товары, 
# с которыми взаимодействовал пользователь
ground_truth = df_test.groupby("user_id")["item_id"].apply(set).to_dict()

In [5]:
# Загружаем тренировочную выборку с CLIP-эмбеддингами (текст + изображение)
df_train = pd.read_csv(
    "data/df_train_CLIP_rating_based.csv",
    na_values=[""],  # исключаем "Unknown"
    keep_default_na=False
)

In [6]:
# Выбираем CLIP-эмбеддинги (текст + изображение)
item_vector_cols = [col for col in df_train.columns if col.startswith("clip_text_") or col.startswith("clip_img_")]
user_ids = df_train["user_id"].unique()

In [7]:
# Извлекаем CLIP-эмбеддинги
clip_vectors  = df_train.drop_duplicates("item_id")[["item_id"] + item_vector_cols].set_index("item_id")

In [8]:
item_vectors = clip_vectors.copy()

In [9]:
print("clip_vectors shape:", clip_vectors.shape)
print("item_vectors shape:", item_vectors.shape)

clip_vectors shape: (67706, 200)
item_vectors shape: (67706, 200)


In [10]:
# Формируем вектор интересов пользователя как среднее по позитивным item-векторам
user_vectors = {}

In [11]:
for user_id, group in tqdm(df_train[df_train["label"] == 1].groupby("user_id"), desc="User vector aggregation", file=sys.stdout):
    item_ids = group["item_id"].values
    vectors = item_vectors.loc[item_ids].values
    user_vectors[user_id] = np.mean(vectors, axis=0)

User vector aggregation: 100%|██████████| 551853/551853 [02:33<00:00, 3605.11it/s]


In [12]:
# Вычислим максимальный Precision@10 для всех юзеров, у которых есть user_vector
def max_possible_precision_at_10(ground_truth, user_vectors, k=10):
    scores = []
    for user_id, gt_items in ground_truth.items():
        if user_id not in user_vectors:
            continue
        max_hits = min(len(gt_items), k)
        scores.append(max_hits / k)
    return round(np.mean(scores), 4)

In [13]:
max_prec = max_possible_precision_at_10(ground_truth, user_vectors)
print("Максимально достижимый Precision@10:", max_prec)

Максимально достижимый Precision@10: 0.1616
